In [ ]:
#! /usr/bin/python

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re as re
import io

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
#import data
import numpy as np 
import pandas as pd
import csv


In [ ]:
#Load Metacyc (pathways) dataframe produced from HumanN2, make sure it is a .csv 

path_data = pd.read_csv("$file_path_to_pathway_data.csv")
print("Metacyc pathways found in metagenomes loaded:")
path_data.head(5)

In [ ]:
#for the metacyc pathways 
path_data['URL.link'] = 'https://biocyc.org/META/NEW-IMAGE?object=' + path_data['ID'].astype(str) 
path_data.head(10)

In [ ]:
# remove Unmapped and unintigrated rows
path_data = path_data[~path_data.ID.str.contains("UNMAPPED")]
path_data = path_data[~path_data.ID.str.contains("UNITEGRATED")]

path_data.head(5)

In [ ]:
#test the URL status to avoid 404 http error
import requests 
from multiprocessing.dummy import Pool as ThreadPool 

urls = path_data['URL.link']

def url_access(x):
    for url in path_data['URL.link']:
        response= requests.get(url)
        status= response.status_code
        path_data['Status']=status

        return requests.head(x).status_code



#use this to run prior code in parallel with multiple threads    
if __name__ == "__main__":
    pool = ThreadPool(4)  # Make the Pool of workers (specific number of threads you want to use)
    results = pool.map(url_access, urls) #Open the urls in their own threads
    pool.close() #close the pool and wait for the work to finish 
    pool.join() 
    

In [ ]:
#This is a check to see if there are any invalid URLs we are looking for nothing in the 400 range
path_data['Status'].value_counts()

In [ ]:
#The heavy lifting code that extracts the taxonomic data. 
import requests
import time
from multiprocessing.dummy import Pool as ThreadPool 

startTime = time.time()


#change to different name of the column that has the URLs
urls = path_data['URL.link']#.head(15)
new_df = []

#this def will run a the following commands when the command get_status is called
def get_status(url):
    try:
        req = Request(url)
        html_page = urlopen(req)
        soup = BeautifulSoup(html_page, "html.parser")
        html_text = soup.get_text()
        f = open("html_text.txt", "w",encoding='utf-8')
        for line in html_text:
            f.write(line)
        f.close()

        html_text = html_text.replace("\n", "") # this removes new lines from metacyc html page making it easier for regex search
        
        pattern = re.findall('{}(.*?){}'.format('Range:', 'Synonyms:'), html_text)
        if len(pattern)==0:# alternate search parameters for pathway data, metacyc htmls are not always unifrom
            pattern = re.findall('{}(.*?){}'.format('Range:', 'BioCyc ID:'), html_text)
            if len(pattern)==0:
                pattern = re.findall('{}(.*?){}'.format('Range:', 'Note'),html_text)  
        new_df.append(pattern) 
        return pattern # this will return the string of interest. 
    except:
        return'Error Invalid URL bypassing, overriding error'
        new_df.append(x)
        pass
if __name__ == "__main__":
    pool = ThreadPool(4)  # Make the Pool of workers (specific number of threads in this case 2)
    results = pool.map(get_status, urls) #Open the urls in their own threads
    pool.close() #close the pool and wait for the work to finish 
    pool.join() 
    
print('Completed multithreaded taxonomic data mininng:\n')
print(len(new_df)) #should match the number of pathways you had in your original data frame.


#outputs how long it took to pull all the taxanomic data
executionTime = (time.time() - startTime)
print('\nExecution time in seconds: ' + str(executionTime)) 

In [ ]:
#Verify output
print(results)
print(len(new_df))
path_data.head(15)

In [ ]:
#pandas function: adds the new_df of the taxa into the existing df while creating new column
df = path_data.assign(taxonomy=new_df)
#df

In [ ]:
#save the dataframe
df.to_csv("$file_path.csv")